# Tutorial 2: OAT15 at high speed stall conditions
## flowTorch workshop 29.09.2025 - 02.10.2025

### Outline
here: 
- loading data from HDF5 instead of OpenFOAM format
- then use different geoemtry objects
- use `N_cells_max` as stopping criterion instead of approximation of the metric

need to have the geometry and data, otherwise replacce with own geometries and data

In [1]:
import sys
import numpy as np
import torch as pt

from stl import mesh
from os.path import join
from os import environ, system

environ["sparseSpatialSampling"] = ".."
sys.path.insert(0, environ["sparseSpatialSampling"])

from sparseSpatialSampling.export import ExportData
from sparseSpatialSampling.sparse_spatial_sampling import SparseSpatialSampling
from sparseSpatialSampling.geometry import CubeGeometry, GeometryCoordinates2D

Refer to the installation instructions at https://github.com/FlowModelingControl/flowtorch
If you are not using the TecplotDataloader, ignore this warning.


In [2]:
def load_airfoil_from_stl_file(_load_path: str, _name: str = "oat15.stl", sf: float = 1.0, dimensions: str = "xy",
                               x_offset: float = 0.0, y_offset: float = 0.0, z_offset: float = 0.0):
    """
    Example function for loading airfoil geometries stored as STL file and extract an enclosed 2D-area from it.
    Important Note:

        the structure of the coordinates within the stl files depends on the order the blocks are exported from
        Paraview; the goal is to form an enclosed area, through which we can draw a polygon. Therefore, the way of
        loading and sorting the data depends on the stl file. For an airfoil, the data can be sorted as:

            TE -> via suction side -> LE -> via pressure side -> TE

        It is helpful to export the airfoil geometry without a training edge and close it manually by connecting the
        last points from pressure to suction side

    :param _load_path: path to the STL file
    :param _name: name of the STL file
    :param sf: scaling factor, in case the airfoil needs to be scaled
    :param dimensions: which plane (orientation) to extract from the STL file
    :param x_offset: offset for x-direction, in case the airfoil should be shifted in x-direction
    :param y_offset: offset for y-direction, in case the airfoil should be shifted in y-direction
    :param z_offset: offset for z-direction, in case the airfoil should be shifted in z-direction
    :return: coordinates representing a 2D-airfoil as enclosed area
    """
    # mapping for the coordinate directions
    dim_mapping = {"x": 0, "y": 1, "z": 2}
    dimensions = [dim_mapping[d] for d in dimensions.lower()]

    # load stl file
    stl_file = mesh.Mesh.from_file(_load_path)

    # scale the airfoil to the original size used in CFD and shift if specified
    stl_file.x = stl_file.x * sf + x_offset
    stl_file.y = stl_file.y * sf + y_offset
    stl_file.z = stl_file.z * sf + z_offset

    # stack the coordinates (zeros column, because values are the same in all columns)
    coord_af = np.stack([stl_file.x[:, 0], stl_file.y[:, 0], stl_file.z[:, 0]], -1)

    # remove duplicates without altering the order -> required, otherwise the number of points is very large
    coord_af = coord_af[:, dimensions]
    _, idx = np.unique(coord_af, axis=0, return_index=True)
    coord_af = coord_af[np.sort(idx)]

    return coord_af

In [3]:
# path to the CFD data and settings
load_path = join("..", "data", "2D", "OAT15")
save_path = join("..", "run", "tutorials", "tutorial_2")

# here we want to use the N_cells_max stopping criterion
n_cells_max = 25000
save_name = f"OAT15_{n_cells_max}_cells"

In [4]:
# load the coordinates of the original grid used in CFD
xz = pt.load(join(load_path, "vertices_and_masks.pt"))

# load the Mach nummber field of the original CFD data
field = pt.load(join("..", "data", "2D", "OAT15", f"ma_large_every10.pt"))

# compute the metric, we want to resolve the buffet, so it make sense to use the std(Ma)
metric = pt.std(field, dim=1)

In [5]:
# load the airfoil geometry of the leading airfoil from an STL file
oat15 = load_airfoil_from_stl_file(join(load_path, "oat15_airfoil_no_TE.stl"), dimensions="xz")

# load the rear NACA airfoil
naca = load_airfoil_from_stl_file(join(load_path, "naca_airfoil_no_TE.stl"), dimensions="xz")

# define the boundaries for the domain and assemble the geometry objects
xz = pt.stack([xz[f"x_large"], xz[f"z_large"]], dim=-1)
bounds = [[pt.min(xz[:, 0]).item(), pt.min(xz[:, 1]).item()], [pt.max(xz[:, 0]).item(), pt.max(xz[:, 1]).item()]]

geometry = [CubeGeometry("domain", True, bounds[0], bounds[1]),
            GeometryCoordinates2D("OAT15", False, oat15, refine=True),
            GeometryCoordinates2D("NACA", False, naca, refine=True)]


In [6]:
# load the corresponding write times
times = pt.load(join(load_path, "oat15_tandem_times.pt"))[::10]

In [7]:
# instantiate an S^3 object
s_cube = SparseSpatialSampling(xz, metric, geometry, save_path, save_name, "OAT15", n_jobs=4, n_cells_max=n_cells_max,
                               write_times=times.tolist())

# execute S^3
s_cube.execute_grid_generation()

[2025-08-14 11:22:07] INFO     
	Selected settings:
		_pre_select          :	False
		_n_jobs              :	4
		_max_delta_level     :	False
		_geometry            :	['domain', 'OAT15', 'NACA']
		_min_metric          :	0.75
		_n_cells_max         :	25000
		_min_level           :	5
		_cells_per_iter_start:	245
		_cells_per_iter_end  :	245
		_cells_per_iter      :	245
		_cells_per_iter_last :	1000000000.0
		_reach_at_least      :	0.75
		_n_dimensions        :	2
		_n_cells_orig        :	245568
		_relTol              :	0.001
[2025-08-14 11:22:07] INFO     Starting refinement:
	Starting iteration no. 0, N_cells = 1
	Starting iteration no. 1, N_cells = 4
	Starting iteration no. 2, N_cells = 8
	Starting iteration no. 3, N_cells = 32
	Starting iteration no. 4, N_cells = 96
[2025-08-14 11:22:12] INFO     Finished uniform refinement.
[2025-08-14 11:22:12] INFO     Starting adaptive refinement.
	Starting iteration no. 0, N_cells = 320
	Starting iteration no. 1, N_cells = 1055
	Starting iteration 

Refer to the installation instructions at https://github.com/FlowModelingControl/flowtorch
If you are not using the TecplotDataloader, ignore this warning.
Refer to the installation instructions at https://github.com/FlowModelingControl/flowtorch
If you are not using the TecplotDataloader, ignore this warning.
Refer to the installation instructions at https://github.com/FlowModelingControl/flowtorch
If you are not using the TecplotDataloader, ignore this warning.
Refer to the installation instructions at https://github.com/FlowModelingControl/flowtorch
If you are not using the TecplotDataloader, ignore this warning.


[2025-08-14 11:22:35] INFO     Finished refinement in 27.5825 s 
								(35 iterations).
								Time for uniform refinement: 5.1138 s
								Time for adaptive refinement: 11.2853 s
								Time for geometry refinement: 6.6494 s
								Time for renumbering the final mesh: 4.5298 s
								
                                    Number of cells: 28919
                                    Minimum ref. level: 6
                                    Maximum ref. level: 12
                                    Captured metric of original grid: 56.29 %
                  


In [8]:
# create export instance, export all fields into the same HFD5 file and create single XDMF from it
# HDF5 may throws an error when running multiple notebooks in parallel. In that case close the otehr notebooks and restart the Kernel
export = ExportData(s_cube)
export.export(xz, field.unsqueeze(1), "Ma")

[2025-08-14 11:22:35] INFO     Starting interpolation and export of field Ma.
[2025-08-14 11:22:39] INFO     Writing HDF5 file for field Ma.
[2025-08-14 11:22:39] INFO     Writing XDMF file for file OAT15_25000_cells.h5
[2025-08-14 11:22:39] INFO     Finished export of field Ma in 4.969s.
